In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
plt.style.use('fivethirtyeight') # For plots

In [2]:
outlet_1 = pd.read_excel('Master Data (Sales).xlsx', sheet_name=0, parse_dates = ['Date of Sale'])
outlet_2 = pd.read_excel('Master Data (Sales).xlsx', sheet_name=1, parse_dates = ['Date of Sale'])
outlet_3 = pd.read_excel('Master Data (Sales).xlsx', sheet_name=2, parse_dates = ['Date of Sale'])

In [3]:
def preprocessing(df, product):
    df = df.copy()
    df.drop(['Product - Chinese Name', 'Product - English Name'], axis=1, inplace=True)
    tmp = df[df['SKU']==product]
    tmp.set_index('Date of Sale', inplace=True)
    tmp.drop('SKU', axis=1, inplace=True)
    idx = pd.date_range('2018-01-01', '2020-12-31')
    tmp = tmp.reindex(idx, fill_value=0)
    return tmp

In [4]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index.
    """
    df = df.copy()
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X

In [5]:
def main_func(df, product):
    df = preprocessing(df, product)
    X, y = create_features(df, label='Amount Sold')
    features_and_target = pd.concat([X, y], axis=1)
    split_date = '2020-11-30'
    df_train = df.loc[df.index <= split_date].copy()
    df_test = df.loc[df.index > split_date].copy()
    model = Prophet()
    model.fit(df_train.reset_index() \
              .rename(columns={'index':'ds',
                               'Amount Sold':'y'}))
    df_test_fcst = model.predict(df=df_test.reset_index() \
                                   .rename(columns={'index':'ds'}))
    return df_test_fcst

In [6]:
def final_function(results, outlet, product):
    results = results[['ds', 'yhat']]
    results['ds'] = results.ds.dt.strftime('%Y/%m/%d')
    results['yhat'] = results['yhat'].round(0)
    results.columns = ['date', 'sales']
    results = results.head(31)
    return results

In [7]:
product_ls = outlet_1['SKU'].unique()

In [8]:
outlet_ls = [outlet_1, outlet_2, outlet_3]

In [10]:
writer = pd.ExcelWriter('sales_predict.xlsx', engine='openpyxl')
for x in range(len(outlet_ls)):
    for y in range(len(product_ls)):
        results = main_func(outlet_ls[x], product_ls[y])
        results = final_function(results, x+1, product_ls[y])
        results['sales'] = results['sales'].clip(lower=0)

        results.to_excel(writer, sheet_name="outlet{}_{}".format(x+1, str(product_ls[y])), index = False)

        writer.save()
        print(str(outlet_ls[x]), product_ls[y], 'done!')

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\YongHao\Anaconda\envs\tf\lib\site-packages\pandas\core\

          SKU Product - English Name Product - Chinese Name Date of Sale  \
0        A013         Mushroom Combo                   菌类拼盘   2018-01-01   
1        A013         Mushroom Combo                   菌类拼盘   2018-01-02   
2        A013         Mushroom Combo                   菌类拼盘   2018-01-03   
3        A013         Mushroom Combo                   菌类拼盘   2018-01-04   
4        A013         Mushroom Combo                   菌类拼盘   2018-01-05   
...       ...                    ...                    ...          ...   
14287  A006-2    Bamboo Shoot (Half)                 笋片（半份）   2020-12-27   
14288  A006-2    Bamboo Shoot (Half)                 笋片（半份）   2020-12-28   
14289  A006-2    Bamboo Shoot (Half)                 笋片（半份）   2020-12-29   
14290  A006-2    Bamboo Shoot (Half)                 笋片（半份）   2020-12-30   
14291  A006-2    Bamboo Shoot (Half)                 笋片（半份）   2020-12-31   

       Amount Sold  
0               14  
1                7  
2                7  
3  

C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\YongHao\Anaconda\envs\tf\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

          SKU Product - English Name Product - Chinese Name Date of Sale  \
0        A013         Mushroom Combo                   菌类拼盘   2018-01-01   
1        A013         Mushroom Combo                   菌类拼盘   2018-01-02   
2        A013         Mushroom Combo                   菌类拼盘   2018-01-03   
3        A013         Mushroom Combo                   菌类拼盘   2018-01-04   
4        A013         Mushroom Combo                   菌类拼盘   2018-01-05   
...       ...                    ...                    ...          ...   
14287  A006-2    Bamboo Shoot (Half)                 笋片（半份）   2020-12-27   
14288  A006-2    Bamboo Shoot (Half)                 笋片（半份）   2020-12-28   
14289  A006-2    Bamboo Shoot (Half)                 笋片（半份）   2020-12-29   
14290  A006-2    Bamboo Shoot (Half)                 笋片（半份）   2020-12-30   
14291  A006-2    Bamboo Shoot (Half)                 笋片（半份）   2020-12-31   

       Amount Sold  
0               14  
1                7  
2                7  
3  

C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\YongHao\Anaconda\envs\tf\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

...

          SKU Product - English Name Product - Chinese Name Date of Sale  \
0      A001-1         Enoki Mushroom                    金针菇   2018-01-01   
1      A001-1         Enoki Mushroom                    金针菇   2018-01-02   
2      A001-1         Enoki Mushroom                    金针菇   2018-01-03   
3      A001-1         Enoki Mushroom                    金针菇   2018-01-04   
4      A001-1         Enoki Mushroom                    金针菇   2018-01-05   
...       ...                    ...                    ...          ...   
13240  A005-2    Buna Shimeiji(Half)                蟹味菇(半份）   2019-06-04   
13241  A005-2    Buna Shimeiji(Half)                蟹味菇(半份）   2019-06-05   
13242  A005-2    Buna Shimeiji(Half)                蟹味菇(半份）   2019-06-07   
13243  A005-2    Buna Shimeiji(Half)                蟹味菇(半份）   2019-06-08   
13244  A005-2    Buna Shimeiji(Half)                蟹味菇(半份）   2019-06-09   

       Amount Sold  
0               26  
1                4  
2                3  
3  

C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\YongHao\Anaconda\envs\tf\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

          SKU Product - English Name Product - Chinese Name Date of Sale  \
0      A001-1         Enoki Mushroom                    金针菇   2018-01-01   
1      A001-1         Enoki Mushroom                    金针菇   2018-01-02   
2      A001-1         Enoki Mushroom                    金针菇   2018-01-03   
3      A001-1         Enoki Mushroom                    金针菇   2018-01-04   
4      A001-1         Enoki Mushroom                    金针菇   2018-01-05   
...       ...                    ...                    ...          ...   
13240  A005-2    Buna Shimeiji(Half)                蟹味菇(半份）   2019-06-04   
13241  A005-2    Buna Shimeiji(Half)                蟹味菇(半份）   2019-06-05   
13242  A005-2    Buna Shimeiji(Half)                蟹味菇(半份）   2019-06-07   
13243  A005-2    Buna Shimeiji(Half)                蟹味菇(半份）   2019-06-08   
13244  A005-2    Buna Shimeiji(Half)                蟹味菇(半份）   2019-06-09   

       Amount Sold  
0               26  
1                4  
2                3  
3  

C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\YongHao\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


          SKU Product - English Name Product - Chinese Name Date of Sale  \
0      A001-1         Enoki Mushroom                    金针菇   2018-01-01   
1      A001-1         Enoki Mushroom                    金针菇   2018-01-02   
2      A001-1         Enoki Mushroom                    金针菇   2018-01-03   
3      A001-1         Enoki Mushroom                    金针菇   2018-01-04   
4      A001-1         Enoki Mushroom                    金针菇   2018-01-05   
...       ...                    ...                    ...          ...   
13240  A005-2    Buna Shimeiji(Half)                蟹味菇(半份）   2019-06-04   
13241  A005-2    Buna Shimeiji(Half)                蟹味菇(半份）   2019-06-05   
13242  A005-2    Buna Shimeiji(Half)                蟹味菇(半份）   2019-06-07   
13243  A005-2    Buna Shimeiji(Half)                蟹味菇(半份）   2019-06-08   
13244  A005-2    Buna Shimeiji(Half)                蟹味菇(半份）   2019-06-09   

       Amount Sold  
0               26  
1                4  
2                3  
3  